       ## ANALYSIS 4TH CAMPAIGN

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ROOT
import numpy as np
import os
from array import array

from utils_lib import  wu_rootfile, wu_rootfileList, fromDatafile_fill

In [3]:
RAW_DATA = "/home/cquintana/Software/Docker/TPA_TCT_root/SiC4thCamp/RawData/"
ROOT_FILES = "/home/cquintana/Software/Docker/TPA_TCT_root/SiC4thCamp/RootFiles/"


FILES = {
    "1MW2": {
        "DEF": [
            {
                "filename": "20250606_1501_1MW2_400nm_350V_reps3_zscan_baseline_substrated",
                "subtype": "zscan",
            },
            {
                "filename": "20250606_1516_1MW2_400nm_450V_reps3_zscan_baseline_substrated",
                "subtype": "zscan",
            },
        ],
    },


}

def rf(file_name):
    return f"{ROOT_FILES}{file_name}.root"

def ls_rootfile(file_name, tree_name="", max_preview=5):
    """
    max_preview:
      - nº máximo de elementos a mostrar si el vector es pequeño
      - si len(vector) > max_preview → solo se imprime el tamaño
    Si tree_name es "" (default), lista todos los TTrees y muestra sus parámetros.
    """
    import ROOT

    def vector_to_list(obj):
        cls = obj.__class__.__name__ if hasattr(obj, "__class__") else ""
        if "vector" in cls:
            return [vector_to_list(v) for v in obj]
        return obj

    def shape_of_list(value):
        shape = []
        cur = value
        while isinstance(cur, list):
            shape.append(len(cur))
            if len(cur) == 0:
                break
            cur = cur[0]
        return shape

    def format_shape(shape):
        if not shape:
            return ""
        return "".join([f"[{d}]" for d in shape])

    def preview_value(value):
        if isinstance(value, list):
            shape = shape_of_list(value)
            shape_str = format_shape(shape)
            n = len(value)
            if n == 0:
                return f"vector{shape_str} (empty)"
            if n <= max_preview:
                return f"vector{shape_str} {value}"
            return f"vector{shape_str}"
        return value

    def print_tree(t, name):
        print(f"\n--- TTree: {name} (estructura) ---")
        t.Print()

        if t.GetEntries() == 0:
            print("[INFO] Tree sin entradas")
            return

        print(f"\n--- TTree: {name} (valores, entry 0) ---")
        t.GetEntry(0)

        for br in t.GetListOfBranches():
            bname = br.GetName()
            val = getattr(t, bname)
            cls = val.__class__.__name__

            # ---------- vectores (incluye anidados) ----------
            if "vector" in cls:
                list_val = vector_to_list(val)
                preview = preview_value(list_val)
                print(f"{bname} = {preview}")

            # ---------- strings ----------
            elif "string" in cls:
                print(f"{bname} = '{str(val)}'")

            # ---------- escalares ----------
            else:
                print(f"{bname} = {val}")

    def list_trees(file_obj):
        trees = []
        for key in file_obj.GetListOfKeys():
            obj = key.ReadObj()
            if obj.InheritsFrom("TTree"):
                trees.append(obj.GetName())
        return trees

    root_path = rf(file_name)

    f = ROOT.TFile.Open(root_path)
    if not f or f.IsZombie():
        print(f"[ERROR] No se pudo abrir: {root_path}")
        return

    print(f"\n=== Contenido de {root_path} ===")
    f.ls()

    if tree_name == "":
        tree_names = list_trees(f)
        if not tree_names:
            print("[WARNING] No hay TTrees en el archivo")
            f.Close()
            return
        for tname in tree_names:
            t = f.Get(tname)
            if t:
                print_tree(t, tname)
        f.Close()
        return

    t = f.Get(tree_name)
    if not t:
        print(f"[WARNING] No existe el TTree '{tree_name}'")
        f.Close()
        return

    print_tree(t, tree_name)
    f.Close()



### ANÁLISIS - 1MW2


#### Generate root files 

In [4]:
IMG_Path = RAW_DATA = "/home/cquintana/Software/Docker/TPA_TCT_root/SiC4thCamp/1MW2_Images/"


In [5]:
from pathlib import Path

filenames = [entry["filename"] for entry in FILES["1MW2"]["DEF"]]
root_files = [ f"{ROOT_FILES}{file_name}.root" for file_name in filenames]
print("DATAFILES:")
for name in filenames:
    outpath = Path(ROOT_FILES) / f"{name}.root"
    print(filenames)
    _nameList = ["dataPath"]
    _dataList = [f"{RAW_DATA}{name}"]
    wu_rootfile(str(outpath), _nameList, _dataList, tree_name="Base")  # si quieres inicializar “vacío”
print("DONE!")

DATAFILES:
['20250606_1501_1MW2_400nm_350V_reps3_zscan_baseline_substrated', '20250606_1516_1MW2_400nm_450V_reps3_zscan_baseline_substrated']
['20250606_1501_1MW2_400nm_350V_reps3_zscan_baseline_substrated', '20250606_1516_1MW2_400nm_450V_reps3_zscan_baseline_substrated']
DONE!


#### Initial fill - WFs + initial parameters



In [6]:
initial_pars = {
  "dt": 0.05,
}

fromDatafile_fill(filenames, ROOT_FILES, RAW_DATA, tree_name="FromFile")

# Common mods:
for key, value in initial_pars.items():
  wu_rootfileList(root_files, [key], [value],  tree_name="Pars")

In [7]:
ls_rootfile(filenames[1])


=== Contenido de /home/cquintana/Software/Docker/TPA_TCT_root/SiC4thCamp/RootFiles/20250606_1516_1MW2_400nm_450V_reps3_zscan_baseline_substrated.root ===

--- TTree: Base (estructura) ---

--- TTree: Base (valores, entry 0) ---
dataPath = '/home/cquintana/Software/Docker/TPA_TCT_root/SiC4thCamp/1MW2_Images/20250606_1516_1MW2_400nm_450V_reps3_zscan_baseline_substrated'

--- TTree: Pars (estructura) ---

--- TTree: Pars (valores, entry 0) ---
dt = 0.05000000074505806
TFile**		/home/cquintana/Software/Docker/TPA_TCT_root/SiC4thCamp/RootFiles/20250606_1516_1MW2_400nm_450V_reps3_zscan_baseline_substrated.root	
 TFile*		/home/cquintana/Software/Docker/TPA_TCT_root/SiC4thCamp/RootFiles/20250606_1516_1MW2_400nm_450V_reps3_zscan_baseline_substrated.root	
  KEY: TTree	Base;1	Analysis base tree (single entry)
  KEY: TTree	Pars;1	Analysis base tree (single entry)
******************************************************************************
*Tree    :Base      : Analysis base tree (single entry) 

In [8]:
for _rp in root_files:
    f = ROOT.TFile.Open(_rp)
    _tree = f.Get("analysis")
    _tree.GetEntry(0)

    # número de samples temporales
    n_samples = int(_tree.WFsRaw_shape1)

    # vectores
    events = np.arange(n_samples, dtype=np.int32)
    t = events.astype(np.float64) * float(_tree.dt)

    f.Close()

    # escribir de vuelta al mismo ROOT
    wu_rootfile(_rp, ["events", "t"], [events, t],  tree_name="Raw")

print("DONE!")


AttributeError: 'TObject' object has no attribute 'GetEntry'

In [ ]:
#ls_rootfile(filenames[1])

plot_root_param_xy(
    out_dir=IMG_Path,
    out_name="t_vs_WFsRaw",
    root_files=[root_files[0]],
    x_param="t",
    y_param="WFsRaw",
    labels=["t vs WFsRaw", "t", "signal"],
    show=True,
    style="line",
)

In [ ]:
# Visual analysis of WFs
_nameList = ["aTBL", "aTLeft", "aTRight"]
_dataList = [25, 26.55, 44.0]
wu_rootfileList(root_files, _nameList, _dataList)

In [ ]:
# Calculation of individual BLLevel TLeft and TRight
for _rp in root_files:
    _BLLevel = 
    _TLeft = 
    _TRight = 

print("DONE!")